In [1]:
import pandas as pd
import numpy as py
import matplotlib.pyplot as plt


In [2]:
# Teammate 0 code begins here
# changes here
state_name_list = [ "Alabama", "Alaska", "Arizona", "Arkansas", "California",
                    "Colorado", "Connecticut", "DC", "Delaware", "Florida",
                    "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana",
                    "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine",
                    "Maryland", "Massachusetts", "Michigan", "Minnesota", 
                    "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada",
                    "New_Hampshire", "New_Jersey", "New_Mexico", "New_York", "North_Carolina",
                    "North_Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania",
                    "Rhode_Island", "South_Carolina", "South_Dakota", "Tennessee", "Texas",
                    "Utah", "Vermont", "Virginia", "Washington", "West_Virginia", 
                    "Wisconsin", "Wyoming"
                  ]

# An empty collection of dataframes
crime_by_state = {}


debug = False


print("Importing *.csv files...")

# Base file path we will use for reusable code
base_file_path = "Raw_Data/CrimeStateByState_"

# Import the U.S. crime data totals
file_path = base_file_path + "US" + ".csv"
if (debug):
    print(f"Importing file: {file_path}")
us_totals_df = pd.read_csv(file_path, header=5)

# Iterate over and import the state-by-state crime data
for state in state_name_list:
    file_path = base_file_path + state + ".csv"
    if (debug):
        print(f"Importing file: {file_path}")
    crime_by_state[state] = pd.read_csv(file_path, header=5)

# Import the gun ownership data
file_path = "Raw_Data/gun_ownership_by_state.csv"
if (debug):
    print(f"Importing file: {file_path}")
gun_ownership_df = pd.read_csv(file_path)

print("File import complete.")

# Clean up the footer information that was at the bottom of each crime data .csv file
if debug:
    print("Cleaning up the .csv files...")
    
for state in state_name_list:
    if debug:
        print(f"The current state is {state}...")
    crime_by_state[state] = crime_by_state[state][crime_by_state[state]["Year"].astype(str).str.isnumeric()]
    
if debug:
    print(f"The current state is {state}...")
us_totals_df = us_totals_df[us_totals_df["Year"].astype(str).str.isnumeric()]

if debug:
    print("Cleanup of .csv files is complete.")

Importing *.csv files...
File import complete.


In [3]:
# Reindex the gun ownership data
try:
    gun_ownership_df.set_index("State", inplace=True)
except:
    pass


In [4]:
all_state_crime_2014 = pd.DataFrame()

# Build a dataframe of the 2014 crime data of all the states and index it to the state name
for state in state_name_list:

    if debug:
        print(f"The current state is {state}...")
    # Give each row the state name so that it can be used as an index later
    crime_by_state[state]["State"] = state
    # Use "this_state" as short-hand
    this_state = crime_by_state[state]
    # Grab the specific row we want
    this_state_crime_2014 = this_state[this_state["Year"] == "2014"]
    # We do this because .append() doesn't support "inplace"
    all_state_crime_2014 = all_state_crime_2014.append(this_state_crime_2014)
    

all_state_crime_2014.set_index(["State"], inplace=True)

try:
    gun_ownership_df = gun_ownership_df.join(all_state_crime_2014, how="outer")
except:
    pass

# Remove bad columns
gun_ownership_df = gun_ownership_df.loc[:, ~gun_ownership_df.columns.str.contains('^Unnamed')]


In [5]:
# dataPoints = ['Violent crime total']
# for col in dataPoints:
#     dat = []
#     for stateName in gun_ownership_df.index:
#         if stateName in crime_by_state:
#             dat.append(crime_by_state[stateName].tail(1)[col].values[0])
#         else:
#             dat.append(None)
#     gun_ownership_df[col] = dat
gun_ownership_df.head(60)

,Rank,# of guns per capita,# of guns registered,Year,Population,Violent crime total,Murder and nonnegligent Manslaughter,Legacy rape /1,Revised rape /2,Robbery,...,Violent Crime rate,Murder and nonnegligent manslaughter rate,Legacy rape rate /1,Revised rape rate /2,Robbery rate,Aggravated assault rate,Property crime rate,Burglary rate,Larceny-theft rate,Motor vehicle theft rate
State,,,,,,,,,,,,,,,,,,,,,
Alabama,6,33.150,161641,2014,4849377,20727,276.0,1436.0,2005.0,4701.0,...,427.4,5.7,29.6,41.3,96.9,283.4,3177.6,819.0,2149.5,209.1
Alaska,15,21.380,15824,2014,736732,4684,41.0,555.0,771.0,629.0,...,635.8,5.6,75.3,104.7,85.4,440.2,2760.0,427.6,2096.4,236.0
Arizona,10,25.610,179738,2014,6731484,26916,319.0,2464.0,3378.0,6249.0,...,399.9,4.7,36.6,50.2,92.8,252.1,3197.5,647.1,2289.1,261.3
Arkansas,8,26.570,79841,2014,2966369,14243,165.0,1182.0,1763.0,2050.0,...,480.1,5.6,39.8,59.4,69.1,346.0,3338.0,835.7,2313.5,188.8
California,44,8.710,344622,2014,38802500,153709,1699.0,8398.0,11527.0,48680.0,...,396.1,4.4,21.6,29.7,125.5,236.6,2441.1,522.3,1527.4,391.3
Colorado,28,16.480,92435,2014,5355866,16554,151.0,2121.0,3039.0,3039.0,...,309.1,2.8,39.6,56.7,56.7,192.8,2530.1,438.2,1857.1,234.8
Connecticut,14,22.960,82400,2014,3596677,8522,86.0,571.0,782.0,3159.0,...,236.9,2.4,15.9,21.7,87.8,125.0,1920.4,332.4,1418.1,169.9
DC,2,68.050,47228,2014,658893,8199,105.0,352.0,472.0,3497.0,...,1244.4,15.9,53.4,71.6,530.7,626.1,5182.5,526.0,4082.3,574.1
Delaware,49,5.040,4852,2014,935614,4576,54.0,249.0,386.0,1269.0,...,489.1,5.8,26.6,41.3,135.6,306.4,2982.0,616.5,2230.1,135.4


In [6]:
# Teammate 0 code ends here

In [7]:
# Teammate 1 code begins here

In [8]:
# Teammate 1 code ends here

In [9]:
# Teammate 2 code begins here

In [10]:
# Teammate 2 code ends here

In [11]:
# Teammate Glenda code begins here

In [13]:
gun_ownership_per_capita_df = gun_ownership_df
gun_ownership_per_capita_df = gun_ownership_per_capita_df.reset_index(drop=False)
gun_ownership_per_capita_df = gun_ownership_per_capita_df[["State","# of guns per capita"]]
gun_ownership_per_capita_df.head()

,State,# of guns per capita
0,Alabama,33.15
1,Alaska,21.38
2,Arizona,25.61
3,Arkansas,26.57
4,California,8.71


In [16]:
all_state_crime_2014_df = all_state_crime_2014
all_state_crime_2014_df = all_state_crime_2014_df.reset_index(drop=False)
all_state_crime_2014_df.head()

,State,Year,Population,Violent crime total,Murder and nonnegligent Manslaughter,Legacy rape /1,Revised rape /2,Robbery,Aggravated assault,Property crime total,...,Murder and nonnegligent manslaughter rate,Legacy rape rate /1,Revised rape rate /2,Robbery rate,Aggravated assault rate,Property crime rate,Burglary rate,Larceny-theft rate,Motor vehicle theft rate,Unnamed: 22
0,Alabama,2014,4849377,20727,276.0,1436.0,2005.0,4701.0,13745.0,154094.0,...,5.7,29.6,41.3,96.9,283.4,3177.6,819.0,2149.5,209.1,NaN
1,Alaska,2014,736732,4684,41.0,555.0,771.0,629.0,3243.0,20334.0,...,5.6,75.3,104.7,85.4,440.2,2760.0,427.6,2096.4,236.0,NaN
2,Arizona,2014,6731484,26916,319.0,2464.0,3378.0,6249.0,16970.0,215240.0,...,4.7,36.6,50.2,92.8,252.1,3197.5,647.1,2289.1,261.3,NaN
3,Arkansas,2014,2966369,14243,165.0,1182.0,1763.0,2050.0,10265.0,99018.0,...,5.6,39.8,59.4,69.1,346.0,3338.0,835.7,2313.5,188.8,NaN
4,California,2014,38802500,153709,1699.0,8398.0,11527.0,48680.0,91803.0,947192.0,...,4.4,21.6,29.7,125.5,236.6,2441.1,522.3,1527.4,391.3,NaN


In [17]:
murder_manslaughter_2014_df = all_state_crime_2014_df[["State","Murder and nonnegligent manslaughter rate"]]
murder_manslaughter_2014_df.head()

,State,Murder and nonnegligent manslaughter rate
0,Alabama,5.7
1,Alaska,5.6
2,Arizona,4.7
3,Arkansas,5.6
4,California,4.4


In [18]:
aggravated_assault_2014_df = all_state_crime_2014_df[["State","Aggravated assault rate"]]
aggravated_assault_2014_df.head()

,State,Aggravated assault rate
0,Alabama,283.4
1,Alaska,440.2
2,Arizona,252.1
3,Arkansas,346.0
4,California,236.6


In [19]:
vehicle_theft_2014_df = all_state_crime_2014_df[["State","Motor vehicle theft rate"]]
vehicle_theft_2014_df.head()

,State,Motor vehicle theft rate
0,Alabama,209.1
1,Alaska,236.0
2,Arizona,261.3
3,Arkansas,188.8
4,California,391.3


In [20]:
robbery_2014_df = all_state_crime_2014_df[["State","Robbery rate"]]
robbery_2014_df.head()

,State,Robbery rate
0,Alabama,96.9
1,Alaska,85.4
2,Arizona,92.8
3,Arkansas,69.1
4,California,125.5


In [64]:
burglary_2014_df = all_state_crime_2014_df[["State","Burglary rate"]]
burglary_2014_df.head()

,State,Burglary rate
0,Alabama,819.0
1,Alaska,427.6
2,Arizona,647.1
3,Arkansas,835.7
4,California,522.3


In [101]:
import plotly.express as px

combined_data_df = pd.merge(burglary_2014_df,gun_ownership_per_capita_df,on="State")

import plotly.express as px

fig = px.scatter(combined_data_df,
                 x="Burglary rate", 
                 y="# of guns per capita",
                 size="Burglary rate", 
                 color="State",
                 hover_name="State",
                 title="Burglary Rate in the US"
                )
fig.show()

In [102]:
import plotly.express as px

combined_data_df = pd.merge(robbery_2014_df,gun_ownership_per_capita_df,on="State")

import plotly.express as px

fig = px.scatter(combined_data_df,
                 x="Robbery rate", 
                 y="# of guns per capita",
                 size="Robbery rate", 
                 color="State",
                 hover_name="State",
                 title="Robbery Rate in the US"
                )
fig.show()

In [103]:
import plotly.express as px

combined_data_df = pd.merge(vehicle_theft_2014_df,gun_ownership_per_capita_df,on="State")

import plotly.express as px

fig = px.scatter(combined_data_df,
                 x="Motor vehicle theft rate", 
                 y="# of guns per capita",
                 size="Motor vehicle theft rate", 
                 color="State",
                 hover_name="State",
                 title="Motor Behicle Theft Rate in the US"
                )
fig.show()

In [104]:
import plotly.express as px

combined_data_df = pd.merge(aggravated_assault_2014_df,gun_ownership_per_capita_df,on="State")

import plotly.express as px

fig = px.scatter(combined_data_df,
                 x="Aggravated assault rate", 
                 y="# of guns per capita",
                 size="Aggravated assault rate", 
                 color="State",
                 hover_name="State",
                 title="Agravated Assault Rate in the US"
                )
fig.show()

In [106]:
import plotly.express as px

combined_data_df = pd.merge(murder_manslaughter_2014_df,gun_ownership_per_capita_df,on="State")

import plotly.express as px

fig = px.scatter(combined_data_df,
                 x="Murder and nonnegligent manslaughter rate", 
                 y="# of guns per capita",
                 size="Murder and nonnegligent manslaughter rate", 
                 color="State",
                 hover_name="State",
                 title="Murder and Manslaughter Rate in the US"
                )
fig.show()

In [97]:
murder_manslaughter_2014_df.head()

,State,Murder and nonnegligent manslaughter rate
0,Alabama,5.7
1,Alaska,5.6
2,Arizona,4.7
3,Arkansas,5.6
4,California,4.4


In [ ]:
# Teammate Glenda code ends here